# Example

A simple text generation model:

In [ ]:
import mlflow
from transformers import pipeline, AutoTokenizer

task = "text-generation"
architecture = "gpt2"

sentence_pipeline = pipeline(
    task=task, tokenizer=AutoTokenizer.from_pretrained(architecture), model=architecture
)

# Validate that the overrides function
prompts = ["Generative models are", "I'd like a coconut so that I can"]

# validation of config prior to save or log
inference_config = {
    "top_k": 2,
    "num_beams": 5,
    "max_length": 30,
    "temperature": 0.62,
    "top_p": 0.85,
    "repetition_penalty": 1.15,
}

# Verify that no exceptions are thrown
sentence_pipeline(prompts, **inference_config)

mlflow.transformers.save_model(
    transformers_model=sentence_pipeline,
    path="my_sentence_generator",
    task=task,
    inference_config=inference_config,
)

## Add content moderation

In [ ]:
from mlspells.operationalize import ResourceAccessMode, ValueSourceType, ValueSource
from mlspells.operationalize.models.moderation import AzureContentSafetyGuardrailModel, ModerationActionBehavior, ModerationAction

Wrap the model with a content safety model:

In [ ]:
moderated = AzureContentSafetyGuardrailModel(
    model_uri="my_sentence_generator", 
    endpoint_uri="https://<my-azure-content-safety-endpoint>.cognitiveservices.azure.com/",
    access_mode=ResourceAccessMode.Key, 
    action=ModerationAction(
        ModerationActionBehavior.Drop, 
        message="Content has been blocked by Azure Content Safety."
    ),
    config={
        "key": ValueSource(source=ValueSourceType.Environment, name="ACS_KEY")
    }
)

Save the new model:

In [ ]:
moderated.save_model("moderated_sentence_generator")